## Загрузка модулей и считывание данных

In [1]:
import osmium as osm
import pandas as pd
import numpy as np
import svgwrite as svg
import random

class Node:
    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon
        self.tags = {}
        self.count = 0
        
class Way:
    def __init__(self):
        self.tag = ""
        self.nodes = []    
        
        
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.nodes = {}
        self.ways = []
        self.hospitals = {}
        
        # Рассчитаем информацию для отрисовки
        self.min_lat = 90
        self.max_lat = -90
        self.min_lon = 180
        self.max_lon = 0
        
    
    def node(self, n):
        x = Node(n.location.lat, n.location.lon)
        
        self.min_lat = min(self.min_lat, n.location.lat)
        self.max_lat = max(self.max_lat, n.location.lat)
        self.min_lon = min(self.min_lon, n.location.lon)
        self.max_lon = max(self.max_lon, n.location.lon)            
        
        is_hospital = False
        for tag in n.tags:
            x.tags[tag.k] = tag.v 
            if tag.v == "clinic" or tag.v == "hospital":
                is_hospital = True
        if is_hospital: 
            x.count+=1
            self.hospitals[n.id] = x
            
        self.nodes[n.id] = x
        
    def way(self, w):
        proceed = False
        for tag in w.tags:
            if tag.k == "highway" and (tag.v == "primary" or tag.v == "secondary" or tag.v == "tertiary" or tag.v == "unclassified"):
                proceed  = True
                way_type = tag.v
        if proceed:
            x = Way()
            x.tag = way_type
            for nd in w.nodes:
                x.nodes.append(nd.ref)
                self.nodes[nd.ref].count+=1   #?
            self.ways.append(x)    

            
def SqrDistance(a, b):
    return (a.lat - b.lat)**2 + (a.lon - b.lon)**2
            

    
osmhandler = OSMHandler()
osmhandler.apply_file("Data/xapi_meta.osm") 

#Удаляем промежуточные узлы в путях
for w in osmhandler.ways:
    w.shortened_nodes = []
    for i, x in enumerate(w.nodes):
        if osmhandler.nodes[x].count > 1 or i == 0 or i == len(w.nodes) - 1:
            w.shortened_nodes.append(x) 
        else:
            del osmhandler.nodes[x] 
            
#Удаляем не встретившиеся узлы
for k in osmhandler.nodes.keys():
    if osmhandler.nodes[k].count == 0:
        del osmhandler.nodes[k]


#Добавляем пути к госпиталям
hospitals = [x for i, x in enumerate(osmhandler.hospitals.keys()) if i < 10]
min_dist = [100 for i in xrange(0,10)]
closest = ["" for i in xrange(0,10)]

for n in osmhandler.nodes.keys():
    for i,x in enumerate(hospitals):
        if SqrDistance(osmhandler.nodes[n], osmhandler.hospitals[x]) < min_dist[i] and n != x:
            min_dist[i] = SqrDistance(osmhandler.nodes[n], osmhandler.hospitals[x])
            closest[i] = n
            
for i,x in enumerate(hospitals):
    w = Way()
    w.nodes.append(hospitals[i])
    w.nodes.append(closest[i])
    w.shortened_nodes = w.nodes
    osmhandler.ways.append(w)

## Алгоритмы поиска пути
### A*

In [2]:
def Manhattan(a, b):
    return abs(a.lat - b.lat) + abs(a.lon - b.lon)

from heapq import heappush, heappop
        
def Astar(data, s, t):
    open_list = []    #heap
    closed_list = []
    g = 0
    f = 0 + Manhattan(data.nodes[s], data.nodes[t])
    heappush(open_list, (f, g, [s])) #загружаем узлы в кучу в виде кортежа 
    while len(open_list)>0:
        turple = heappop(open_list)
        p = turple[-1] #выгружаем из минимального кортежа путь
        g = turple[1]
        x = p[-1]
        if x in closed_list:
            continue
        if x == t:
            return (f, p)
        closed_list.append(x)
        #
        for successor in adjacency_list[x]:
            ng = g + SqrDistance(data.nodes[successor], data.nodes[x])
            nf = ng + Manhattan(data.nodes[successor], data.nodes[t])**2
            new_path = p[:]
            new_path.append(successor)
            heappush(open_list, (nf, ng, new_path))
    return []  


### Дейкстра

In [3]:
def Dijkstra(data, s):
    dist = {s:0}
    prev = {}
    
    seen = set()
    
    for n in data.nodes.keys():
        if n != s:
            dist[n] = 100 #inf
    
    q = [] #heap
    heappush(q, (0, s))
    
    while len(q)>0:
        turple = heappop(q)
        x = turple[1]
        if x in seen:
            continue
        
        seen.add(x)
        for successor in adjacency_list[x]:
            alt = dist[x] + SqrDistance(data.nodes[x], data.nodes[successor])**(0.5) #?
            if alt < dist[successor]:
                dist[successor] = alt
                prev[successor] = x
                heappush(q, (alt, successor))
    
    return prev  #, dist

### Левит

In [4]:
def Levit(data, s):    # Модифицированная версия с неявным хранением очередей
    dist = {s:0}
    prev = {}
    
    for n in data.nodes.keys():
        if n != s:
            dist[n] = 100 #inf
            
    q = [] #heap
    heappush(q, (0, s))
    
    while len(q)>0:
        turple = heappop(q)
        x = turple[1]
        
        for successor in adjacency_list[x]:
            alt = dist[x] + SqrDistance(data.nodes[x], data.nodes[successor])**(0.5) #?
            if alt < dist[successor]:
                dist[successor] = alt
                prev[successor] = x
                heappush(q, (alt, successor))
    
    return prev

## Интерфейс для ввода координат

In [5]:
print ('Введите значение в пределах: lat ({0}, {1}); lon ({2}, {3})'.format(osmhandler.min_lat, osmhandler.max_lat, osmhandler.min_lon, osmhandler.max_lon))
lat, lon = map(float, raw_input().split())
start = Node(lat, lon)
osmhandler.nodes[0] = start

# Добавляем путь к стартовой позиции
for n in osmhandler.nodes.keys():
    if SqrDistance(osmhandler.nodes[n], start) < min_dist and n != 0:
        min_dist = SqrDistance(osmhandler.nodes[n], start)
        closest = n
w = Way()
w.nodes.append(0)
w.nodes.append(closest)
w.shortened_nodes = w.nodes
osmhandler.ways.append(w)

Введите значение в пределах: lat (56.6991083, 56.9448682); lon (60.2707188, 60.941436)
56.7 60.5


In [6]:
edge_list = []

for w in osmhandler.ways:
    for n in xrange(len(w.shortened_nodes) - 1):
        x = w.shortened_nodes[n]
        y = w.shortened_nodes[n + 1]
        edge_list.append([x,y])
        
adjacency_list = {}
for n in osmhandler.nodes.keys():
    adjacent_nodes = []
    for e in edge_list:
        if e[0] == n:
            adjacent_nodes.append(e[1])
        if e[1] == n:
            adjacent_nodes.append(e[0])
    adjacency_list[n] = adjacent_nodes

## Тестирование быстродействия и оптимальности

In [36]:
import time

starting_points = [random.choice(osmhandler.nodes.keys()) for x in xrange(100)]

# Dijkstra
t = time.clock()
for start in starting_points:
    prev = Dijkstra(osmhandler, start);
    path1 = []
    x = hospitals[0]
    while x != start:
        path1.append(x)
        if x not in prev.keys():
            break
        x = prev[x]
    path1.reverse()

print ('Быстродействие Дейкстры на {0} итерациях = {1:.3f} c'.format(len(starting_points), time.clock() - t))

# A star
t = time.clock()
for start in starting_points:
    path2 = Astar(osmhandler, start, hospitals[0])
    if path2 != []:
        path2 = path2[-1]
        
print ('Быстродействие A* на {0} итерациях = {1:.3f} c'.format(len(starting_points), time.clock() - t))

# Levit
t = time.clock()
for start in starting_points:
    prev = Levit(osmhandler, start);
    path3 = []
    x = hospitals[0]
    while x != start:
        path3.append(x)
        if x not in prev.keys():
            break
        x = prev[x]
    path3.reverse()

print ('Быстродействие Левита на {0} итерациях = {1:.3f} c'.format(len(starting_points), time.clock() - t))

Быстродействие Дейкстры на 100 итерациях = 1.060 c
Быстродействие A* на 100 итерациях = 0.065 c
Быстродействие Левита на 100 итерациях = 1.084 c


In [35]:
error_Dijkstra, error_A, error_Levit = 0,0,0
n = 0
for start in starting_points:
    # Dijkstra
    found_path = True
    prev = Dijkstra(osmhandler, start);
    path1 = []
    x = hospitals[0]
    while x != start:
        path1.append(x)
        if x not in prev.keys():
            found_path = False
            break
        x = prev[x]
    path1.reverse()
    if not found_path:
        continue
    
    # A star
    path2 = Astar(osmhandler, start, hospitals[0])
    if path2 != []:
        path2 = path2[-1]
    else:
        continue
    
    # Levit
    found_path = True
    prev = Levit(osmhandler, start);
    path3 = []
    x = hospitals[0]
    while x != start:
        path3.append(x)
        if x not in prev.keys():
            found_path = False
            break
        x = prev[x]
    path3.reverse()
    if not found_path:
        continue    
    
    f1, f2, f3 = 0, 0, 0  
    for i in xrange (0, len(path1)-1):
        f1 += SqrDistance(osmhandler.nodes[path1[i]], osmhandler.nodes[path1[i+1]])**(0.5)  #dijkstra
    for i in xrange (0, len(path2)-1):
        f2 += SqrDistance(osmhandler.nodes[path2[i]], osmhandler.nodes[path2[i+1]])**(0.5)  #a*
    for i in xrange (0, len(path3)-1):
        f3 += SqrDistance(osmhandler.nodes[path3[i]], osmhandler.nodes[path3[i+1]])**(0.5)  #levit
    
    n+=1
    f = min(f1,f2,f3)
    error_Dijkstra += f1/f - 1
    error_A += f2/f - 1
    error_Levit += f3/f - 1
    
error_Dijkstra *= 100/n
error_A *= 100/n 
error_Levit *= 100/n

print ('Средняя ошибка Дейкстры = {0:.1f}%; А* = {1:.1f}%; Левита = {2:.1f}%'.format(error_Dijkstra, error_A, error_Levit))

Средняя ошибка Дейкстры = 0.0%; А* = 16.8%; Левита = 0.0%


## SVG

In [9]:
# Строим минимальные пути от введенной позиции до больниц
paths = []
path_lens = []
for x in hospitals:
    path = Astar(osmhandler, 0, x)
    if path != []:
        paths.append( path[-1] )
        path_lens.append( path[0] )
ind_of_shortest = path_lens.index(min(path_lens)) 

In [17]:
bot = osmhandler.min_lat
left = osmhandler.min_lon
up = osmhandler.max_lat
right = osmhandler.max_lon

scale = 1000

dwg = svg.Drawing('map_with_path.svg')
for e in edge_list:
    x1 = (osmhandler.nodes[e[0]].lat - bot)*scale
    y1 = (osmhandler.nodes[e[0]].lon - left)*scale
    x2 = (osmhandler.nodes[e[1]].lat - bot)*scale
    y2 = (osmhandler.nodes[e[1]].lon - left)*scale
    dwg.add(dwg.line((x1, y1), (x2, y2), stroke=svg.rgb(10, 10, 16, '%'), stroke_width=0.5))

# Рисуем пути
for j,path in enumerate(paths):  
    if j != ind_of_shortest:     
        for i in xrange(0, len(path) - 1):
            x1 = (osmhandler.nodes[path[i]].lat - bot)*scale
            y1 = (osmhandler.nodes[path[i]].lon - left)*scale
            x2 = (osmhandler.nodes[path[i+1]].lat - bot)*scale
            y2 = (osmhandler.nodes[path[i+1]].lon - left)*scale
            dwg.add(dwg.line((x1, y1), (x2, y2), stroke=svg.rgb(100 - 10*j , 100 + 10*j, 16, '%'), stroke_width=0.7 - 0.02*j))

# Рисуем кратчайший путь
path = paths[ind_of_shortest]
for i in xrange(0, len(path) - 1):
    x1 = (osmhandler.nodes[path[i]].lat - bot)*scale
    y1 = (osmhandler.nodes[path[i]].lon - left)*scale
    x2 = (osmhandler.nodes[path[i+1]].lat - bot)*scale
    y2 = (osmhandler.nodes[path[i+1]].lon - left)*scale
    dwg.add(dwg.line((x1, y1), (x2, y2), stroke=svg.rgb(200, 10, 10, '%'), stroke_width=0.7))

dwg.save()